In [1]:
from __future__ import print_function
import sys
import io
import random
import numpy as np
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import RMSprop

Using TensorFlow backend.
F:\Installed\Anaconda3\envs\tf_gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
F:\Installed\Anaconda3\envs\tf_gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
F:\Installed\Anaconda3\envs\tf_gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
F:\Installed\Anaconda3\envs\tf_gpu\lib\site-packag

In [2]:
text = open('song_lyrics.txt', 'r').read().lower()
print('Corpus Length:', len(text))
text[:500]

Corpus Length: 196350


'"hot air balloon"\n\nwe wrote a prelude to our own fairytale\nand bought a parachute at a church rummage sale.\nand with a mean sewing machine and miles of thread\nwe sewed the day above l.a. in navy and red.\n\nwe round a racetrack through your mom\'s kitchen chairs\nand fought the shadows back down your dark basement stairs.\ni lit a match and let it catch to light up the room.\nand then you yelled as we beheld an old maroon.\n\nhot air balloon.\n\ni\'ll be out of my mind,\nand you\'ll be out of ideas pretty so'

In [3]:
chars = sorted(list(set(text)))
print('Total Chars:', len(chars))
print(chars)
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

Total Chars: 61
['\n', ' ', '!', '"', '&', "'", '(', ')', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '8', '9', ':', ';', '?', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '¢', '¦', '©', '¼', 'â', 'ã', '˜', '€', '™']


In [4]:
# Cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('Number of Sequences:', len(sentences))
print(sentences[:5])

Number of Sequences: 65437
['"hot air balloon"\n\nwe wrote a prelude to', 't air balloon"\n\nwe wrote a prelude to ou', 'ir balloon"\n\nwe wrote a prelude to our o', 'balloon"\n\nwe wrote a prelude to our own ', 'loon"\n\nwe wrote a prelude to our own fai']


In [5]:
# Vectorization
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

In [6]:
# Creating the LSTM model
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))
optimizer = RMSprop(0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Instructions for updating:
Colocations handled automatically by placer.


In [7]:
def sample(preds, temperature=1.0):
    # Helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [8]:
def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print('--------------------------------------')
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- Diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=15,
          callbacks=[print_callback])

Instructions for updating:
Use tf.cast instead.
Epoch 1/15
65437/65437 [==============================] - 46s 701us/step - loss: 2.2104
--------------------------------------
----- Generating text after Epoch: 0
----- Diversity: 0.2
----- Generating with seed: "year old kid from saint paul
hopped on a"
year old kid from saint paul
hopped on and in the right to the say
i to the the the and and in the fream the made to the say ing the say
in the and and in the bark and and and and in the say the the songht in the freet to the sing and my and the sing to and in the frown and i take and in the the the the the me the the start and in the cars to the fream to the the the the the sheres and in the cares in the the start in the and the start 
----- Diversity: 0.5
----- Generating with seed: "year old kid from saint paul
hopped on a"
year old kid from saint paul
hopped on a toring the the pream and your they to the fart tranday then when the gorthing to the bast
and in the fornd is the reaping 

i stoll my doess new ore the songs with a final staring, downs
do you say-oh, i am all my down
i love the she chands on the wings home and way, burn
so i still the pleating down
and i a manin to do
i say to sea

i loved and 
----- Diversity: 1.0
----- Generating with seed: "was lying on the couch and there's this "
was lying on the couch and there's this live out oro
you wear, whan's light so ime touch, some that'l to sea
you corich topise your veny sonigi
all of moldy grow ringsing as whist out
the horosu seach to youe
and just thasing sundorb of rostinima down
and tulpar juve and i agh
i can't wake itbees off
a sing down my fowds as the chade
stop of m a lone
the farith a hoak as my new lest brocks
i suve in your breath might sime
to treesillssi
----- Diversity: 1.2
----- Generating with seed: "was lying on the couch and there's this "
was lying on the couch and there's this kneps
on cho iston't off nottangeg, snow whete's refs
what a side i'll have love whisisyins sougs
it's do you 

65437/65437 [==============================] - 48s 738us/step - loss: 1.2115
--------------------------------------
----- Generating text after Epoch: 7
----- Diversity: 0.2
----- Generating with seed: "show until you've finally flown away

yo"
show until you've finally flown away

you were the one and the stars to real ratched out
that we were the one and i wanna creat back to the sky
and we the sky to metroon
i staying and i wanna creat to say
it spend my had i was spream
i still the store
i'll never know than a shooting star
so i was driving me crazy)
(da-da-da-da-da-da-da-da-da-da-da-da-da-da-da-da-da-da-da-da-da-da-da-da-da-da-da-da-da-da-da-da-da-da-da-da-da-da-da-da-da-
----- Diversity: 0.5
----- Generating with seed: "show until you've finally flown away

yo"
show until you've finally flown away

you wanna cry and then i was a beart of the grace of sountillawner and we're been heart would you got to mettic day and let me so the sky
we can still the speed of a partice to try
is 

shine, shine, shine from the caasin missing for veres
i can't fly don't sed new blue soarning grot were

dearf (no one the city
(oh halgha)
'cause we this is hope back to cloping live
'cause i hopechat you 
here's "bome and the atmy birsing cold !

i blound an ore
held to washer out
and for around
when you're here gownt bottline with youâ€™ll be out an the hill' be illa.
so iz oull then i am hot so you donn
and coun
----- Diversity: 1.2
----- Generating with seed: "sun,
starting today,
shine, shine, shine"
sun,
starting today,
shine, shine, shine your breitmas it wart
hay she know pit the chanted through, tore buls, you're holiving to where wight

"her ounsiof in i lopecsed and ice true
saild winnays down wenlerres
on the deeming burn.
(her hopeshoolvere to the sky,
keemanti )
abour of first clip zy by eyes just when uping at them

rightared all my mindo
will shiutro gle
anore
you onft cup my hand autake it
every fleard to dresem home,
sa
Epoch 12/15
65437/65437 [======================

F:\Installed\Anaconda3\envs\tf_gpu\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


ause i love in the alligator sky

i won't feel alive
like the cold when i was go
i am a shour of the rock of a sloes and the way

"up it like the fire the fire the rest and there's a moment a shour of the ree
mall be a hout of sing of the way

when the way, it was always a good time


----- Diversity: 0.5
----- Generating with seed: " said and done
we'll shine like the sun
"
 said and done
we'll shine like the sun
i love the back and i'm through the way

i left my hunds
this is driving me crazy
this is always a good time

all i need a hour in the dark and right and i am a stleas
'cause this is out of cancee up to me
i left my hunds a slour of a silve
it's all i knew the way!
but i'm slowang for the dreams and changer to shine no matter
year through the come the brest and the cold all fling we close you're o
----- Diversity: 1.0
----- Generating with seed: " said and done
we'll shine like the sun
"
 said and done
we'll shine like the sun

from chanightlrocky braver
lot me so you got the

In [9]:
# Serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk
